In [154]:
from importlib import reload
import data.loader as loader_module
import evaluation.predict as predict_module
import evaluation.eval as eval_module
import data.squad_loader as squad_loader_module
import json


reload(predict_module)
reload(eval_module)
reload(loader_module)
reload(squad_loader_module)

from evaluation.predict import ModelPredictor
from evaluation.eval import evaluate_generation, evaluate_classification, get_confusion_matrix
from data.loader import Dataset, get_dataset
from data.squad_loader import get_squad_dataset

## Load dataset

In [172]:
full_evaluation = True
train_set, test_set = get_squad_dataset(
    500 if full_evaluation else 50,
    split_percentage=1.0,
    percent_types=(1.0, 0.0, 0.0),
    types=("answerable", "unanswerable_ner", "unanswerable_base"),
)
print(len(train_set))
print(train_set[-1])
train_set.data[-2:]


500
{'context': 'Pāśupata Shaivism (Pāśupata, "of Paśupati") is the oldest of the major Shaiva schools. The philosophy of Pashupata sect was systematized by Lakulish in the 2nd century CE. Paśu in Paśupati refers to the effect (or created world), the word designates that which is dependent on something ulterior. Whereas, Pati means the cause (or principium), the word designates the Lord, who is the cause of the universe, the pati, or the ruler. Pashupatas disapproved of Vaishnava theology, known for its doctrine servitude of souls to the Supreme Being, on the grounds that dependence upon anything could not be the means of cessation of pain and other desired ends. They recognised that those depending upon another and longing for independence will not be emancipated because they still depend upon something other than themselves. According to Pāśupatas, soul possesses the attributes of the Supreme Deity when it becomes liberated from the \'germ of every pain\'.', 'question': 'What is the 

[{'context': 'The ancestors of modern bacteria were unicellular microorganisms that were the first forms of life to appear on Earth, about 4 billion years ago. For about 3 billion years, most organisms were microscopic, and bacteria and archaea were the dominant forms of life. In 2008, fossils of macroorganisms were discovered and named as the Francevillian biota. Although bacterial fossils exist, such as stromatolites, their lack of distinctive morphology prevents them from being used to examine the history of bacterial evolution, or to date the time of origin of a particular bacterial species. However, gene sequences can be used to reconstruct the bacterial phylogeny, and these studies indicate that bacteria diverged first from the archaeal/eukaryotic lineage. Bacteria were also involved in the second great evolutionary divergence, that of the archaea and eukaryotes. Here, eukaryotes resulted from the entering of ancient bacteria into endosymbiotic associations with the ancestors of 

In [156]:
for i in range(min(len(train_set), 10)):
    print(train_set.data[i])


{'context': 'Following the 11 September 2001 attacks, former President of Pakistan Pervez Musharraf sided with the U.S. against the Taliban government in Afghanistan after an ultimatum by then U.S. President George W. Bush. Musharraf agreed to give the U.S. the use of three airbases for Operation Enduring Freedom. United States Secretary of State Colin Powell and other U.S. administration officials met with Musharraf. On 19 September 2001, Musharraf addressed the people of Pakistan and stated that, while he opposed military tactics against the Taliban, Pakistan risked being endangered by an alliance of India and the U.S. if it did not cooperate. In 2006, Musharraf testified that this stance was pressured by threats from the U.S., and revealed in his memoirs that he had "war-gamed" the United States as an adversary and decided that it would end in a loss for Pakistan.', 'question': "Who said he had 'war-gamed' the US?", 'id': '572848bfff5b5019007da0e2', 'answers': [{'text': 'Musharraf',

## Predict

In [150]:
#model_name = "davinci-002"
model_name = "gpt-3.5-turbo-instruct"
model = ModelPredictor(model_name)

In [93]:
def extract_unknown(text):
    if "no answer" in text.lower():
        return "" # evaluation process wants empty string for predictions of unanswerable questions
    else:
        return text


In [94]:
"""CONTEXT: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
QUESTION: What century did the Normans first gain their separate identity?
ANSWER(concise): 10th

CONTEXT: Building construction is the process of adding structure to real property or construction of buildings. The majority of building construction jobs are small renovations, such as addition of a room, or renovation of a bathroom. Often, the owner of the property acts as laborer, paymaster, and design team for the entire project. Although building construction projects typically include various common elements, such as design, financial, estimating and legal considerations, many projects of varying sizes reach undesirable end results, such as structural collapse, cost overruns, and/or litigation. For this reason, those with experience in the field make detailed plans and maintain careful oversight during the project to ensure a positive outcome.
QUESTION: Who acts as laborer, paymaster, and design team for a renovation project?
ANSWER(concise): the owner

CONTEXT: Pathogens can rapidly evolve and adapt, and thereby avoid detection and neutralization by the immune system; however, multiple defense mechanisms have also evolved to recognize and neutralize pathogens. Even simple unicellular organisms such as bacteria possess a rudimentary immune system, in the form of enzymes that protect against bacteriophage infections. Other basic immune mechanisms evolved in ancient eukaryotes and remain in their modern descendants, such as plants and invertebrates. These mechanisms include phagocytosis, antimicrobial peptides called defensins, and the complement system. Jawed vertebrates, including humans, have even more sophisticated defense mechanisms, including the ability to adapt over time to recognize specific pathogens more efficiently. Adaptive (or acquired) immunity creates immunological memory after an initial response to a specific pathogen, leading to an enhanced response to subsequent encounters with that same pathogen. This process of acquired immunity is the basis of vaccination.
QUESTION: The immune systems of bacteria have enzymes that protect against infection by what kind of cells?
ANSWER(concise): bacteriophage

CONTEXT: {context}
QUESTION: {question}
ANSWER(concise): """ 

'Given the following context:\nCONTEXT: {context}\nAnswer the following question:\nQUESTION: {question}\nANSWER: '

In [139]:
prompt_template = """CONTEXT: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
QUESTION: What century did the Normans first gain their separate identity?
ANSWER(concise): 10th

CONTEXT: Building construction is the process of adding structure to real property or construction of buildings. The majority of building construction jobs are small renovations, such as addition of a room, or renovation of a bathroom. Often, the owner of the property acts as laborer, paymaster, and design team for the entire project. Although building construction projects typically include various common elements, such as design, financial, estimating and legal considerations, many projects of varying sizes reach undesirable end results, such as structural collapse, cost overruns, and/or litigation. For this reason, those with experience in the field make detailed plans and maintain careful oversight during the project to ensure a positive outcome.
QUESTION: Who acts as laborer, paymaster, and design team for a renovation project?
ANSWER(concise): the owner

CONTEXT: Pathogens can rapidly evolve and adapt, and thereby avoid detection and neutralization by the immune system; however, multiple defense mechanisms have also evolved to recognize and neutralize pathogens. Even simple unicellular organisms such as bacteria possess a rudimentary immune system, in the form of enzymes that protect against bacteriophage infections. Other basic immune mechanisms evolved in ancient eukaryotes and remain in their modern descendants, such as plants and invertebrates. These mechanisms include phagocytosis, antimicrobial peptides called defensins, and the complement system. Jawed vertebrates, including humans, have even more sophisticated defense mechanisms, including the ability to adapt over time to recognize specific pathogens more efficiently. Adaptive (or acquired) immunity creates immunological memory after an initial response to a specific pathogen, leading to an enhanced response to subsequent encounters with that same pathogen. This process of acquired immunity is the basis of vaccination.
QUESTION: The immune systems of bacteria have enzymes that protect against infection by what kind of cells?
ANSWER(concise): bacteriophage

CONTEXT: {context}
QUESTION: {question}
ANSWER(concise): """ 

results_generate, original_preds = model.generate(
    dataset=train_set,
    prompt_template=prompt_template,
    extract_function=lambda text: text, #extract_unknown, 
    num_samples=1,
    max_tokens=50,
)
results_generate, original_preds

([' General Steinmetz',
  ' disease',
  '1930s',
  'RMS St Helena',
  'Northeast Africa',
  '20th century',
  ' Cushitic',
  " Apple's iPad tablet computer",
  '1977',
  ' Arabic',
  '555 nanometres',
  'charity matches',
  ' To preserve apostolic succession',
  ' large-scale raids',
  "TuiA'ana",
  '1925',
  'oral contraceptives',
  ' Andhra region',
  '16th century',
  ' Yankee Stadium and Ebbets Field.',
  ' Hampshire.',
  '884 million dollars',
  'Aston Martin DB10 and Jaguar C-X75',
  '1066',
  'nearby municipalities in the State of Mexico',
  ' Falaise and then Rouen',
  'the monarch',
  ' lectures',
  ' William Robertson Coe',
  '2014',
  'adornment',
  '7.6%',
  'A rise from sea level into the upper regions of the atmosphere.',
  '2012',
  ' Ceramics and cement.',
  'polymyxins',
  'no Spring break',
  ' Muslim',
  'South African Rand',
  ' May 2005',
  ' a wide range of activities',
  " country's constitutional monarch",
  ' northern England',
  'vibration and mu',
  ' Multipl

In [105]:
#results_generate = [result.split('\n')[0] for result in original_preds]

In [173]:
def extract_yes_or_no(text):
    if "yes" in text.lower():
        return "Yes"
    elif "no" in text.lower():
        return "No"
    else:
        return None

classify_with_generation_template = """CONTEXT: {context}
QUESTION: {question}

Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): """

results_classify, original_preds_class = model.generate(
    dataset=train_set,
    prompt_template=classify_with_generation_template,
    extract_function=extract_yes_or_no, 
    num_samples=1,
    max_tokens=5,
)
results_classify, original_preds_class

## Eval

In [39]:
#list(train_set.get_column("answers")), results_generate, results_classify

In [131]:
def save_results(prompt, exact_score, f1_score, model_name, full_results=False):
    with open("results/results_squad.txt", "a") as f:
        prepared_prompt = prompt.replace("\n", r"\n")
        f.write(f"{exact_score}\t{f1_score}\t{model_name}\t{full_results}\t{prepared_prompt}\n")

In [120]:
# def load_results():
#     with open("results/results_squad.txt", "r") as f:
#         lines = f.readlines()
#         lines = [line.strip() for line in lines]
#         results = [line.split("\t") for line in lines]
#         return results
# results_squad = load_results()

# mod_res = [res[1:] + (['False'] if len(res)==4 else []) + [res[0] + "\n"] for res in results_squad]
# mod_res

# # save mod_res to file
# with open("results/results_squad.txt", "w") as f:
#     for res in mod_res:
#         f.write("\t".join(res))

In [140]:
import evaluation.squad_eval as squad_eval_module
reload(squad_eval_module)
from evaluation.squad_eval import get_raw_scores_mod

preds = dict(zip(train_set.get_column("id"), results_generate))
raws_scores, f1_scores = get_raw_scores_mod(train_set, preds)

#raws_scores, f1_scores

In [141]:
exact_score = sum(raws_scores.values()) / len(raws_scores)
f1_score = sum(f1_scores.values()) / len(f1_scores)
print("Exact score:", exact_score)
print("F1 score:", f1_score)
save_results(prompt_template, exact_score, f1_score, model_name, full_results=full_evaluation)

Exact score: 0.608
F1 score: 0.7699707826057675


In [157]:
to_choice_mapping = {
    True: "No", # true is_impossible
    False: "Yes", # false is_impossible
}
results_classify, list(train_set.get_type(to_choice_mapping))

(['Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  None,
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 ['Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'])

In [171]:
correct, accuracy = evaluate_classification(
    train_set,
    results_classify, # type: ignore
    to_choice_mapping=to_choice_mapping,
)
print(accuracy)

save_results(classify_with_generation_template, accuracy, accuracy, model_name, full_results=full_evaluation)

0.98


In [19]:
get_confusion_matrix(
    predict=["unknown", "10", "11", "9", "unknown"],
    original=["unknown", "9", "11", "unknown", "unknown"],
    unknown_value = "unknown"
)

(array([[1., 0.],
        [0., 2.],
        [1., 1.]]),
 0)